In [1]:
scPileupVars = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/needle_post/CHIP_b1_Flt3l/old_v01_scPileupVars/"
#scPileupVars = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/needle_post/peaks_chip_genes/CHIP_b1_Control/concat_scPileupVars/pileup.tsv"
vcf = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/variants.all.vcf"
sample = "CHIP_b1_Flt3l"
outdir = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/needle_post/CHIP_b1_Flt3l/cells_vars/"
barcodes_f = "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/CHIP_dec172021_b1/MTBlacklist_A2/Flt3l/outs/filtered_peak_bc_matrix/barcodes.tsv"
#genome_fa = "/data/Mito_Trace/data/processed/genomes/mtMasked/GRCh38_MT_blacklist_A2_2020/fasta/genome.fa"

In [2]:
from glob import glob
from os.path import basename, join
from src.utils.data_io import read_csv_multichar
import pandas as pd
from pandarallel import pandarallel

fig_utils from mplh


In [3]:
pandarallel.initialize(nb_workers=16)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
region_f = vcf+".region.txt"
vcf_df = read_csv_multichar(vcf, sep="\t", multicomment="##")
vcf_df[["#CHROM", "POS"]].to_csv(region_f, index=None, sep="\t", header=None)

In [5]:
vcf_df = vcf_df.loc[vcf_df["#CHROM"].str.contains("^chr")]
vcf_df

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,CHIP_b1_Control,...,CHIP_b1_Input,CHIP_b2_Control,CHIP_b2_Flt3l,CHIP_b2_Input,CHIP_b3_Control,CHIP_b3_Flt3l,CHIP_b3_Input,CHIP_b4_Control,CHIP_b4_Flt3l,CHIP_b4_Input
1000,chr1,10018,.,C,G,97.47380,.,TYPE=snv;NS=12;AF=0.08333333;DP=4744;RO=4722;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/0:0.2387064:681:678:1:0.001468429:402,276,1,...",...,"0/1:97.4738:1249:1237:12:0.009607686:740,497,7...","0/0:0:234:233:0:0:113,120,0,0:-1:-1:0:944.2852:.","0/0:0:225:225:0:0:113,112,0,0:-1:-1:0:926.5604:.","0/0:0:343:343:0:0:176,167,0,0:-1:-1:0:1432.95:.","0/0:16.98098:186:184:2:0.01075269:100,84,0,2:9...","0/0:0:269:269:0:0:130,139,0,0:-1:-1:0:1116.99:.","0/0:0:108:108:0:0:53,55,0,0:-1:-1:0:416.3386:.","0/0:21.71786:356:353:3:0.008426966:162,191,2,1...","0/0:0:315:314:0:0:160,154,0,0:-1:-1:0:1305.324:.","0/0:0:118:118:0:0:67,51,0,0:-1:-1:0:458.7915:."
1001,chr1,10031,.,T,TA,65.50475,.,TYPE=ins;NS=12;AF=0.08333333;DP=6095;RO=6077;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/1:65.50475:856:853:5:0.005841121:532,321,3,2...",...,"0/0:0:1640:1639:0:0:1029,610,0,0:-1:-1:0:Inf:.","0/0:0:315:315:0:0:163,152,0,0:-1:-1:0:1790.363:.","0/0:0:286:286:0:0:146,140,0,0:-1:-1:0:1616.495:.","0/0:0:432:431:0:0:237,194,0,0:-1:-1:0:2483.383:.","0/0:0:245:245:0:0:141,104,0,0:-1:-1:0:1386.389:.","0/0:0:357:357:0:0:189,168,0,0:-1:-1:0:2052.156:.","0/0:0:141:141:0:0:70,71,0,0:-1:-1:0:744.5862:.","0/0:0:446:444:0:0:230,214,0,0:-1:-1:0:2570.377:.","0/0:0:381:379:0:0:208,171,0,0:-1:-1:0:2196.873:.","0/0:0:154:153:0:0:94,59,0,0:-1:-1:0:833.2017:."
1002,chr1,10039,.,AC,A,87.23672,.,TYPE=del;NS=12;AF=0.08333333;DP=6678;RO=6620;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/0:0:946:939:0:0:633,306,0,0:-1:-1:0:Inf:.",...,"0/1:87.23672:1822:1808:8:0.004390779:1257,551,...","0/0:0:341:340:0:0:195,145,0,0:-1:-1:0:1873.462:.","0/0:0:308:306:0:0:178,128,0,0:-1:-1:0:1679.652:.","0/0:0:472:470:0:0:277,193,0,0:-1:-1:0:2615.803:.","0/0:0:259:259:0:0:168,91,0,0:-1:-1:0:1403.992:.","0/0:22.05355:368:361:2:0.005434783:218,143,0,2...","0/0:0:161:160:0:0:94,66,0,0:-1:-1:0:819.4477:.","0/0:21.33713:492:489:2:0.004065041:288,201,2,0...","0/0:0:422:422:0:0:251,171,0,0:-1:-1:0:2310.822:.","0/0:0:176:174:0:0:124,50,0,0:-1:-1:0:933.9806:."
1003,chr1,10051,.,A,ACCG,67.72425,.,TYPE=ins;NS=12;AF=0.08333333;DP=7730;RO=7680;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/0:0:1123:1117:0:0:754,363,0,0:-1:-1:0:Inf:.",...,"0/1:67.72425:2162:2147:6:0.002775208:1497,650,...","0/0:0:384:382:0:0:230,152,0,0:-1:-1:0:2286.389:.","0/0:0:348:348:0:0:212,136,0,0:-1:-1:0:2077.343:.","0/0:0:535:532:0:0:317,215,0,0:-1:-1:0:Inf:.","0/0:0:293:291:0:0:191,100,0,0:-1:-1:0:1747.041:.","0/0:0:419:416:0:0:272,144,0,0:-1:-1:0:2530.343:.","0/0:0:190:190:0:0:113,77,0,0:-1:-1:0:1078.799:.","0/0:0:562:561:0:0:346,215,0,0:-1:-1:0:Inf:.","0/0:0:456:456:0:0:289,167,0,0:-1:-1:0:2738.056:.","0/0:0:208:206:0:0:137,69,0,0:-1:-1:0:1201.061:."
1004,chr1,10055,.,TA,T,60.01402,.,TYPE=del;NS=12;AF=0.1666667;DP=7734;RO=7677;AO...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/1:60.01402:1122:1113:9:0.00802139:707,406,4,...",...,"0/0:0:2135:2124:1:0.0004683841:1442,682,1,0:99...","0/0:0:389:388:0:0:236,152,0,0:-1:-1:0:1570.522:.","0/0:0:352:352:0:0:216,136,0,0:-1:-1:0:1430.16:.","0/0:0.5660855:535:535:1:0.001869159:314,221,0,...","0/0:0:295:294:0:0:196,98,0,0:-1:-1:0:1183.517:.","0/0:0.5660855:422:421:1:0.002369668:270,151,0,...","0/0:0:189:186:0:0:107,79,0,0:-1:-1:0:730.9136:.","0/0:0:557:555:0:0:335,220,0,0:-1:-1:0:2283.826:.","0/0:0:451:449:0:0:288,161,0,0:-1:-1:0:1836.398:.","0/0:0:198:197:0:0:128,69,0,0:-1:-1:0:772.6831:."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6200,chrY,56767254,.,T,A,56.78491,.,TYPE=snv;NS=12;AF=0.08333333;DP=1759;RO=1744;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/0:20.33698:408:405:3:0.007352941:277,128,0,3...",...,"0/0:0:380:380:0:0:252,128,0,0:-1:-1:0

In [6]:
with open(barcodes_f, 'r') as f:
    lines = f.readlines()
barcodes = [x.strip() for x in lines]
barcodes

['AAACGAAAGGAGACTC-1',
 'AAACGAAAGTCACGCC-1',
 'AAACGAACAAGACTAA-1',
 'AAACGAACAAGCGAAC-1',
 'AAACGAACACAGCCAC-1',
 'AAACGAACACAGGTAG-1',
 'AAACGAACACAGTAGG-1',
 'AAACGAACATAGAATG-1',
 'AAACGAACATGTAGAA-1',
 'AAACGAAGTATTCGAC-1',
 'AAACGAAGTCTGATTG-1',
 'AAACGAATCGCTGATA-1',
 'AAACGAATCTCTTAAC-1',
 'AAACGAATCTGTCGTC-1',
 'AAACTCGAGACACAAT-1',
 'AAACTCGAGATGTTCC-1',
 'AAACTCGAGGCACCTC-1',
 'AAACTCGAGGGCTTCC-1',
 'AAACTCGAGTACCCAT-1',
 'AAACTCGAGTCTCCAA-1',
 'AAACTCGAGTGCCCTG-1',
 'AAACTCGAGTGTCCCG-1',
 'AAACTCGCAGGACTGA-1',
 'AAACTCGCAGTAACGG-1',
 'AAACTCGCATATACGC-1',
 'AAACTCGCATCCGTGG-1',
 'AAACTCGCATTAACGC-1',
 'AAACTCGGTAGGTCTT-1',
 'AAACTCGGTATTCTCT-1',
 'AAACTCGGTCAGCTTA-1',
 'AAACTCGGTGAGTCGA-1',
 'AAACTCGGTGCATCAT-1',
 'AAACTCGTCAAGGCCC-1',
 'AAACTCGTCATACTTC-1',
 'AAACTCGTCCGAGAGA-1',
 'AAACTCGTCCGGAAAG-1',
 'AAACTCGTCGATGTGT-1',
 'AAACTCGTCGCGCCAA-1',
 'AAACTCGTCTTACGGA-1',
 'AAACTCGTCTTTGCAT-1',
 'AAACTGCAGCGAGCTA-1',
 'AAACTGCAGTGAAGGA-1',
 'AAACTGCAGTGCTCGC-1',
 'AAACTGCCA

In [7]:
pileup_files = glob(scPileupVars+"/*.bq.tsv")
len(pileup_files)

167288

In [8]:
from tqdm.notebook import tqdm

In [9]:
def is_var(ser, vcf_df, pad=-1):
    if len(vcf_df[(vcf_df["#CHROM"] == ser["chr"]) & (vcf_df["POS"] == ser["pos"]+pad) & (vcf_df["ALT"] == ser["nt"])])>0:
        return True
    return False



In [12]:
barcode_files = []

for cell_f in tqdm(pileup_files):
    cb = basename(cell_f).split("pileups")[0].replace("CB_","")
    #print('cb', cb)
    curr_name = cb.split("-1")[0]
    if (cb in barcodes):
        barcode_files.append(cell_f)
print(len(barcode_files))

  0%|          | 0/167288 [00:00<?, ?it/s]

7651


In [13]:
barcodes_df = pd.DataFrame(barcode_files)

In [15]:
all_cells_vars = []
all_cells_dp = []
for cell_f in tqdm(barcode_files):
    cb = basename(cell_f).split("pileups")[0].replace("CB_","")
    #print('cb', cb)
    curr_name = cb.split("-1")[0]
    if (cb in barcodes):
        #print('in barcodes')
        curr_pile = pd.read_csv(cell_f, sep='\t', index_col=0)
        curr_pile = curr_pile.rename({"nt_x":"nt"}, axis=1)

        #curr_pile["is_var"] = curr_pile.apply(is_var, vcf_df=vcf_df, axis=1)
        curr_pile["cell"] = curr_name
        #curr_vars = curr_pile[curr_pile["is_var"]]
        #all_cells_vars.append(curr_vars)
        all_cells_vars.append(curr_pile)

        curr_dp = curr_pile.groupby(["chr", "pos"]).sum()["count"].reset_index()
        curr_dp["ID"] = curr_dp["chr"] + "_" + curr_dp["pos"].astype(str)
        curr_dp["cell"] = curr_name
        all_cells_dp.append(curr_dp)
    #break

  0%|          | 0/7651 [00:00<?, ?it/s]

In [16]:
len(all_cells_dp)

7651

In [17]:
len(all_cells_vars)

7651

In [18]:
vcf_df.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,CHIP_b1_Control,...,CHIP_b1_Input,CHIP_b2_Control,CHIP_b2_Flt3l,CHIP_b2_Input,CHIP_b3_Control,CHIP_b3_Flt3l,CHIP_b3_Input,CHIP_b4_Control,CHIP_b4_Flt3l,CHIP_b4_Input
1000,chr1,10018,.,C,G,97.47380,.,TYPE=snv;NS=12;AF=0.08333333;DP=4744;RO=4722;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/0:0.2387064:681:678:1:0.001468429:402,276,1,...",...,"0/1:97.4738:1249:1237:12:0.009607686:740,497,7...","0/0:0:234:233:0:0:113,120,0,0:-1:-1:0:944.2852:.","0/0:0:225:225:0:0:113,112,0,0:-1:-1:0:926.5604:.","0/0:0:343:343:0:0:176,167,0,0:-1:-1:0:1432.95:.","0/0:16.98098:186:184:2:0.01075269:100,84,0,2:9...","0/0:0:269:269:0:0:130,139,0,0:-1:-1:0:1116.99:.","0/0:0:108:108:0:0:53,55,0,0:-1:-1:0:416.3386:.","0/0:21.71786:356:353:3:0.008426966:162,191,2,1...","0/0:0:315:314:0:0:160,154,0,0:-1:-1:0:1305.324:.","0/0:0:118:118:0:0:67,51,0,0:-1:-1:0:458.7915:."
1001,chr1,10031,.,T,TA,65.50475,.,TYPE=ins;NS=12;AF=0.08333333;DP=6095;RO=6077;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/1:65.50475:856:853:5:0.005841121:532,321,3,2...",...,"0/0:0:1640:1639:0:0:1029,610,0,0:-1:-1:0:Inf:.","0/0:0:315:315:0:0:163,152,0,0:-1:-1:0:1790.363:.","0/0:0:286:286:0:0:146,140,0,0:-1:-1:0:1616.495:.","0/0:0:432:431:0:0:237,194,0,0:-1:-1:0:2483.383:.","0/0:0:245:245:0:0:141,104,0,0:-1:-1:0:1386.389:.","0/0:0:357:357:0:0:189,168,0,0:-1:-1:0:2052.156:.","0/0:0:141:141:0:0:70,71,0,0:-1:-1:0:744.5862:.","0/0:0:446:444:0:0:230,214,0,0:-1:-1:0:2570.377:.","0/0:0:381:379:0:0:208,171,0,0:-1:-1:0:2196.873:.","0/0:0:154:153:0:0:94,59,0,0:-1:-1:0:833.2017:."
1002,chr1,10039,.,AC,A,87.23672,.,TYPE=del;NS=12;AF=0.08333333;DP=6678;RO=6620;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/0:0:946:939:0:0:633,306,0,0:-1:-1:0:Inf:.",...,"0/1:87.23672:1822:1808:8:0.004390779:1257,551,...","0/0:0:341:340:0:0:195,145,0,0:-1:-1:0:1873.462:.","0/0:0:308:306:0:0:178,128,0,0:-1:-1:0:1679.652:.","0/0:0:472:470:0:0:277,193,0,0:-1:-1:0:2615.803:.","0/0:0:259:259:0:0:168,91,0,0:-1:-1:0:1403.992:.","0/0:22.05355:368:361:2:0.005434783:218,143,0,2...","0/0:0:161:160:0:0:94,66,0,0:-1:-1:0:819.4477:.","0/0:21.33713:492:489:2:0.004065041:288,201,2,0...","0/0:0:422:422:0:0:251,171,0,0:-1:-1:0:2310.822:.","0/0:0:176:174:0:0:124,50,0,0:-1:-1:0:933.9806:."
1003,chr1,10051,.,A,ACCG,67.72425,.,TYPE=ins;NS=12;AF=0.08333333;DP=7730;RO=7680;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/0:0:1123:1117:0:0:754,363,0,0:-1:-1:0:Inf:.",...,"0/1:67.72425:2162:2147:6:0.002775208:1497,650,...","0/0:0:384:382:0:0:230,152,0,0:-1:-1:0:2286.389:.","0/0:0:348:348:0:0:212,136,0,0:-1:-1:0:2077.343:.","0/0:0:535:532:0:0:317,215,0,0:-1:-1:0:Inf:.","0/0:0:293:291:0:0:191,100,0,0:-1:-1:0:1747.041:.","0/0:0:419:416:0:0:272,144,0,0:-1:-1:0:2530.343:.","0/0:0:190:190:0:0:113,77,0,0:-1:-1:0:1078.799:.","0/0:0:562:561:0:0:346,215,0,0:-1:-1:0:Inf:.","0/0:0:456:456:0:0:289,167,0,0:-1:-1:0:2738.056:.","0/0:0:208:206:0:0:137,69,0,0:-1:-1:0:1201.061:."
1004,chr1,10055,.,TA,T,60.01402,.,TYPE=del;NS=12;AF=0.1666667;DP=7734;RO=7677;AO...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/1:60.01402:1122:1113:9:0.00802139:707,406,4,...",...,"0/0:0:2135:2124:1:0.0004683841:1442,682,1,0:99...","0/0:0:389:388:0:0:236,152,0,0:-1:-1:0:1570.522:.","0/0:0:352:352:0:0:216,136,0,0:-1:-1:0:1430.16:.","0/0:0.5660855:535:535:1:0.001869159:314,221,0,...","0/0:0:295:294:0:0:196,98,0,0:-1:-1:0:1183.517:.","0/0:0.5660855:422:421:1:0.002369668:270,151,0,...","0/0:0:189:186:0:0:107,79,0,0:-1:-1:0:730.9136:.","0/0:0:557:555:0:0:335,220,0,0:-1:-1:0:2283.826:.","0/0:0:451:449:0:0:288,161,0,0:-1:-1:0:1836.398:.","0/0:0:198:197:0:0:128,69,0,0:-1:-1:0:772.6831:."


In [22]:
all_cells_vars_df = pd.concat(all_cells_vars)
all_cells_vars_df.head()

,chr,pos,nt,count,nt_y,BQ,cell
0,chr1,67685236,A,1,A,37.0,GACCCAGTCCGTGCGA
1,chr1,67685237,A,1,A,37.0,GACCCAGTCCGTGCGA
2,chr1,67685238,T,1,T,25.0,GACCCAGTCCGTGCGA
3,chr1,67685239,T,1,T,37.0,GACCCAGTCCGTGCGA
4,chr1,67685240,A,1,A,37.0,GACCCAGTCCGTGCGA


In [23]:
all_cells_vars_df["ID"] = all_cells_vars_df.apply(lambda x: f"{x['chr']}_{x['pos']}>{x['nt']}", axis=1)

In [24]:
print(all_cells_vars_df.shape)
print(all_cells_vars_df.duplicated(subset=["cell", "ID"], keep=False).sum())
print(all_cells_vars_df.duplicated(subset=["cell", "ID", "BQ"], keep=False).sum())
print(all_cells_vars_df.duplicated(keep=False).sum())

(161481534, 8)
4133022
1863186
0


## Drop duplicates
(shouldn't be any, need to fix upstream script)

In [65]:
dups = all_cells_vars_df.loc[(all_cells_vars_df.duplicated(subset=["cell", "ID"], keep=False))].sort_values(["ID", "cell"])
dups

,chr,pos,nt,count,nt_y,BQ,is_var,cell,ID


In [66]:
dups = all_cells_vars_df.loc[(all_cells_vars_df.duplicated(subset=["cell", "ID", "BQ"], keep=False))].sort_values(["ID", "cell"])
dups

,chr,pos,nt,count,nt_y,BQ,is_var,cell,ID


In [67]:
all_cells_vars_df = all_cells_vars_df.loc[~(all_cells_vars_df.duplicated(subset=["cell", "ID"], keep='first'))]

In [68]:
ad = all_cells_vars_df.pivot(index="cell", columns="ID", values="count").fillna(0).astype(int)
ad_vars = [x.split(">")[0] for x in ad.columns]
ad

ID,chr10_102245332>A,chr10_10266>A,chr10_10357>A,chr10_10407>C,chr10_110077048>T,chr10_110919124>G,chr10_110919142>C,chr10_110919158>G,chr10_110919434>T,chr10_11163371>G,...,chrY_11312604>T,chrY_11327754>G,chrY_11747705>A,chrY_56676150>C,chrY_56680575>A,chrY_56685916>A,chrY_56706124>T,chrY_56724767>G,chrY_56724805>A,chrY_56768566>A
cell,,,,,,,,,,,,,,,,,,,,,
AAACGAAAGGAGACTC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACGAACATGTAGAA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACTCGAGGCACCTC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACTCGAGTGCCCTG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACTCGCATCCGTGG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTAGTTTACGC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGGTTCAGGTGGTA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGGTTGTACATGGG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
# for ind, val in enumerate(all_cells_dp):
#     all_cells_dp[ind]["cell"] = all_cells_vars[ind].iloc[0]["cell"]

In [94]:
all_cells_dp_df = pd.concat(all_cells_dp)
all_cells_dp_df.head()

,chr,pos,count,ID,cell
0,GL000195.1,30832,1,GL000195.1_30832,GACCCAGTCCGTGCGA
1,GL000195.1,30833,1,GL000195.1_30833,GACCCAGTCCGTGCGA
2,GL000195.1,30834,1,GL000195.1_30834,GACCCAGTCCGTGCGA
3,GL000195.1,30835,1,GL000195.1_30835,GACCCAGTCCGTGCGA
4,GL000195.1,30836,1,GL000195.1_30836,GACCCAGTCCGTGCGA


## Add coverage to all_cells_vars_df

In [123]:
all_cells_vars_df

,chr,pos,nt,count,nt_y,BQ,is_var,cell,ID
463,chr11,16738561,T,2,T,37.000000,True,GACCCAGTCCGTGCGA,chr11_16738561>T
600,chr11,64185847,A,4,A,37.000000,True,GACCCAGTCCGTGCGA,chr11_64185847>A
605,chr11,64185852,T,4,T,37.000000,True,GACCCAGTCCGTGCGA,chr11_64185852>T
622,chr11,64185869,A,4,A,37.000000,True,GACCCAGTCCGTGCGA,chr11_64185869>A
1359,chr13,18212229,A,1,A,37.000000,True,GACCCAGTCCGTGCGA,chr13_18212229>A
...,...,...,...,...,...,...,...,...,...
18462,chrM,16239,A,14,A,36.142857,True,ACTAACGCAACGACAG,chrM_16239>A
18487,chrM,16264,A,13,A,37.000000,True,ACTAACGCAACGACAG,chrM_16264>A
18519,chrM,16296,T,12,T,36.000000,True,ACTAACGCAACGACAG,chrM_16296>T
18578,chrM,16355,T,17,T,36.294118,True,ACTAACGCAACGACAG,chrM_16355>T


In [128]:
len(all_cells_vars_df["ID"].unique())

1542

In [125]:
def get_vars_depth(ser):
    curr_var_pos = ser["ID"].split(">")[0]
    curr_dp = all_cells_dp_df.loc[(all_cells_dp_df["ID"]==curr_var_pos) & ((all_cells_dp_df["cell"]==ser["cell"]))]
    if len(curr_dp) > 1:
        print("dp more than 1", ser)
    if len(curr_dp) == 0:
        print("dp not here?. returning 0", ser)
        return 0
    return ser["count"]/curr_dp.iloc[0]["count"]

all_cells_vars_dp_df = all_cells_vars_df.apply(get_vars_depth, axis=1)
all_cells_vars_dp_df

KeyboardInterrupt: 

In [126]:
all_cells_vars_dp_df

NameError: name 'all_cells_vars_dp_df' is not defined

In [95]:
# dp = all_cells_dp_df.loc[all_cells_dp_df["ID"].isin(ad_vars)].pivot(index="cell", columns="ID", values="count").fillna(0)

# dp.head()

In [120]:
curr_ad_vars = []
for x in ad.columns:
    if x.split(">")[0] in dp.columns:
        curr_ad_vars.append(x)
ad = ad.loc[dp.index, curr_ad_vars]

curr_ad_vars = []
for x in dp.columns:
    if x in ad_vars:
        curr_ad_vars.append(x)
    
        
ad = ad.loc[dp.index, curr_ad_vars]
ad.shape

KeyError: "None of [Index(['chr10_102245332', 'chr10_10266', 'chr10_10357', 'chr10_10407',\n       'chr10_110077048', 'chr10_110919124', 'chr10_110919142',\n       'chr10_110919158', 'chr10_110919434', 'chr10_11163371',\n       ...\n       'chrY_11312604', 'chrY_11327754', 'chrY_11747705', 'chrY_56676150',\n       'chrY_56680575', 'chrY_56685916', 'chrY_56706124', 'chrY_56724767',\n       'chrY_56724805', 'chrY_56768566'],\n      dtype='object', name='ID', length=1534)] are in the [columns]"

In [117]:
dp.shape

(1530, 1534)

In [115]:
ad.values/dp.values

ValueError: operands could not be broadcast together with shapes (1530,1542) (1530,1534) 

In [114]:
dp.sum()

ID
chr10_102245332     65.0
chr10_10266          1.0
chr10_10357        103.0
chr10_10407         68.0
chr10_110077048     75.0
                   ...  
chrY_56685916        1.0
chrY_56706124        1.0
chrY_56724767        8.0
chrY_56724805       12.0
chrY_56768566        3.0
Length: 1534, dtype: float64

In [81]:
#af = ad.apply(lambda x: x/all_cells_dp_df.loc[x.name.split(">")[0], "count"])
ad.head().apply(lambda x: x/all_cells_dp_df.loc[x.name.split(">")[0], "count"])

KeyboardInterrupt: 

In [21]:
# all_cells_vars = []
# all_cells_dp = []
# for cell_f in tqdm(pileup_files):
#     cb = basename(cell_f).split("pileups")[0].replace("CB_","")
#     #print('cb', cb)
#     curr_name = cb.split("-1")[0]
#     if (cb in barcodes):
#         #print('in barcodes')
#         curr_pile = pd.read_csv(cell_f, sep='\t', index_col=0)
#         curr_pile = curr_pile.rename({"nt_x":"nt"}, axis=1)

#         curr_pile["is_var"] = curr_pile.parallel_apply(is_var, vcf_df=vcf_df, axis=1)
#         curr_pile["cell"] = curr_name
#         curr_vars = curr_pile[curr_pile["is_var"]]
#         all_cells_vars.append(curr_vars)

#         curr_dp = curr_pile.groupby(["chr", "pos"]).sum()["count"].reset_index()
#         curr_dp["ID"] = curr_dp["chr"] + "_" + curr_dp["pos"].astype(str)
#         all_cells_dp.append(curr_dp)
    

  0%|          | 0/167288 [00:00<?, ?it/s]

in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in barcodes
in b

Process ForkPoolWorker-6363:
Process ForkPoolWorker-6360:
Process ForkPoolWorker-6361:
Process ForkPoolWorker-6362:
Process ForkPoolWorker-6370:
Process ForkPoolWorker-6372:
Process ForkPoolWorker-6364:
Process ForkPoolWorker-6366:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/i

KeyboardInterrupt: 

  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
Process ForkPoolWorker-6365:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/h

  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/site-packages/pandas/core/apply.py", line 812, in apply_standard
    results, res_index = self.apply_series_generator()
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/site-packages/pandas/core/apply.py", line 812, in apply_standard
    results, res_index = self.apply_series_generator()
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/site-packages/pandarallel/data_types/dataframe.py", line 31, in worker
    return df.apply(func, *args, **kwargs)
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/site-packages/pandas/core/apply.py", line 828, in apply_series_generator
    results[i] = self.f(v)
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/site-packages/pandas/core/apply.py", line 131, in f
    return func(x, *args, **kwargs)
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/site-packages/pandas/core/apply.py", line 828, in apply_series_generator
    results[i] = self.f(v)
  File "/h

  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/site-packages/pandas/_config/config.py", line 115, in _get_option
    key = _get_single_key(pat, silent)
Traceback (most recent call last):
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/isaac/miniconda3/envs/mttrace/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
 

## Create AD matrix with variants

In [ ]:
cells_dp_df = pd.concat(all_cells_dp, ignore_index=True)
cells_vars_df = pd.concat(all_cells_vars, ignore_index=True)

## Save

In [ ]:
cells_dp_df.to_csv(join(outdir, "DP.tsv"))
cells_vars_df.to_csv(join(outdir, "AD.tsv"))